## Importando librerías

In [ ]:
# eliminar warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# modulo para entrenar y testear
from sklearn.model_selection import train_test_split
# modulo para escalamiento o normalizacion
from sklearn.preprocessing import MinMaxScaler
# modulo para analisis estadistico
import statsmodels.api as sm
# VIF detector de colinealidad
from statsmodels.stats.outliers_influence import variance_inflation_factor
# RFE eliminar recursividad y regresion lineal
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression